# 1. Word Embeddings

### 1.1. Токенизация: 

Первый шаг для задач NLP — это разбиение исходных данных на слова.
Текст, с которым мы работаем, представлен в сыром формате: со всей пунктуацией и смайликами, присоединёнными к некоторым словам, поэтому простой str.split не подойдет.

Давайте использовать `nltk` — библиотеку, которая решает множество задач NLP, таких как токенизация, стемминг или определение частей речи.

In [1]:
# !pip install --upgrade nltk gensim bokeh umap-learn numpy

In [1]:
import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

In [2]:
# download the data:
!wget "https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1" -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

Файл «./quora.txt» уже существует; не загружается.


In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [4]:
tokenizer = WordPunctTokenizer()
print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


Приведите все к нижнему регистру и получите токены с помощью токенизатора. `data_tok` должен представлять собой список списков токенов для каждой строки в `data`.

In [6]:
data_tok = # YOUR CODE

In [ ]:
" ".join(data_tok[0])

In [8]:
assert all(
    isinstance(row, (list, tuple)) for row in data_tok
), "please convert each line into a list of tokens (strings)"
assert all(
    all(isinstance(tok, str) for tok in row) for row in data_tok
), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all("a" <= x.lower() <= "z" for x in tok)
assert all(
    map(lambda l: not is_latin(l) or l.islower(), map(" ".join, data_tok))
), "please make sure to lowercase the data"

### 1.2. Word vectors: обучение векторных представлений

Существует не один способ обучения векторных представлений слов. Есть `Word2Vec` и `GloVe` с разными функциями потерь. Также есть `FastText`, который использует символьные модели для обучения эмбеддингов.

Выбор огромен, поэтому давайте начнем с чего-то простого: `gensim` — это еще одна библиотека для NLP, которая включает множество векторных моделей, в том числе `word2vec`.

Этот код создает и обучает модель Word2Vec для получения векторных представлений слов.

In [9]:
from gensim.models import Word2Vec


model = Word2Vec(
    data_tok,        # токенизированные данные
    vector_size=32,  # размерность вектора эмбеддинга
    min_count=5,     # учитывать слова, встречающиеся минимум 5 раз
    window=5,        # размер окна контекста (5 слов вокруг целевого слова)
).wv

In [ ]:
# Теперь вы можете получать векторные представления слов
model.get_vector("anything")

In [ ]:
# Или напрямую запрашивать похожие слова. Поэкспериментируйте с этим!
model.most_similar("bread")

In [ ]:
king = # YOUR CODE
man = # YOUR CODE
woman = # YOUR CODE
result_vector = # YOUR CODE

similar_words = model.similar_by_vector(result_vector, topn=10)
similar_words

### 1.3. Использование предобученной модели


Долго же это заняло, да? А теперь представьте обучение полноразмерных (100~300 измерений) векторных представлений на гигабайтах текста: статьях из википедии или твитах.

К счастью, сегодня вы можете получить предобученную модель векторных представлений всего в 2 строчки кода (без смс, обещаю).

In [13]:
import gensim.downloader as api

model = api.load("glove-twitter-25")

In [ ]:
len(model.key_to_index.keys())

In [15]:
model.sort_by_descending_frequency()

In [ ]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

In [ ]:
model.most_similar(positive=["king", "woman"], negative=["man"])

### 1.4. Визуализация векторных представлений слов

Один из способов проверить качество наших векторов - это построить их визуализацию. Проблема в том, что эти векторы находятся в 30+ мерном пространстве, а мы, люди, больше привыкли к 2-3 измерениям.

К счастью, мы, специалисты по машинному обучению, знаем о методах __понижения размерности__.

Давайте используем их для построения 1000 самых частых слов

In [ ]:
# Получите 1000 самых частоых слов

# key_to_index - словарь в моделях gensim, который 
#   cопоставляет каждому слову (ключ - key) его числовой индекс (value - index)

# keys - метод словаря в Python, который возвращает все ключи словаря

words = # YOUR CODE
print(words[::101])

In [ ]:
 # для каждого слова вычислите его вектор с помощью модели
word_vectors = # YOUR CODE

# Переведите list в numpy array с помощью np.asarray
word_vectors = # YOUR CODE

# Выведите размерность пооучившегося word_vectors
print(# YOUR CODE )

assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 25)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA



Простейшим методом линейного снижения размерности является Метод Главных Компонент (PCA = Principial Component Analysis).

В геометрической интерпретации PCA пытается найти оси, вдоль которых наблюдается наибольшая дисперсия данных. Если угодно, "естественные" оси.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


На математическом уровне метод пытается разложить объектно-признаковую матрицу $X$ на две матрицы меньшей размерности: $W$ и $\hat W$, минимизируя среднеквадратичную ошибку:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - матрица объектов (центрированная);
- $W \in \mathbb{R}^{m \times d}$ - матрица прямого преобразования;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - матрица обратного преобразования;
- $n$ объектов, $m$ исходных измерений и $d$ целевых измерений;



In [20]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Стандартизовываем вектора, иначе PCA будет искажено признаками с большим масштабом.
scaler = StandardScaler() 
word_vectors_pca = scaler.fit_transform(word_vectors)

pca = PCA(2) # объект PCA, который будет сжимать данные до 2 компонент
word_vectors_pca = pca.fit_transform(word_vectors_pca)

In [21]:
assert word_vectors_pca.shape == (
    len(word_vectors),
    2,
), "there must be a 2d vector for each word"

In [ ]:
plt.scatter(word_vectors_pca[:, 0], word_vectors_pca[:, 1])

Не очень информативно...

Давайте сделаем визуализацию красивее с помощью bokeh!
Теперь график стал интерактивным.

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook

output_notebook()
def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs
):
    """draws an interactive plot for data points with auxilirary info on hover"""
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

In [24]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# Наведите курсор мыши на точки и посмотрите, 
# сможете ли вы идентифицировать кластеры

#### Визуализация соседних точек с помощью UMAP
PCA — это хороший метод, но он строго линейный и поэтому способен улавливать только общую высокоуровневую структуру данных.

Если мы хотим сосредоточиться на сохранении близости соседних точек, мы можем использовать UMAP, который сам по себе является методом embedding. Здесь вы можете прочитать [подробнее о UMAP (на русском)](https://habr.com/ru/company/newprolab/blog/350584/) и о [t-SNE](https://distill.pub/2016/misread-tsne/), который также является методом embedding.

In [25]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_vectors)

In [ ]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=words)

# наведите курсор мыши на это место и посмотрите, 
# сможете ли вы идентифицировать кластеры

# 2. Word2vec на PyTorch

### 2.1. Токенизация и формирование датасета

Как вы уже могли заметить, идея, лежащая в основе [word2vec](https://arxiv.org/pdf/1310.4546), достаточно общая. В данном задании вы реализуете его самостоятельно.

Дисклеймер: не стоит удивляться тому, что реализация от `gensim` (или аналоги) обучается быстрее и работает точнее. Она использует множество доработок и ускорений, а также достаточно эффективный код. Ваша задача добиться промежуточных результатов за разумное время.

P.s. Как ни странно, GPU в этом задании нам не потребуется.

Если вы работаете локально, выполните в выбранном окружении следующую команду:

```pip install --upgrade nltk bokeh umap-learn```

In [1]:
!pip install --upgrade nltk bokeh umap-learn

In [2]:
import itertools
import random
import string
from collections import Counter
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import umap
from IPython.display import clear_output
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm.auto import tqdm as tqdma

In [3]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

In [ ]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

In [ ]:
tokenizer = WordPunctTokenizer()
print(tokenizer.tokenize(data[50]))

In [ ]:
data_tok = [tokenizer.tokenize(
        line.translate(str.maketrans("", "", string.punctuation)).lower()
    )
    for line in data
]
len(data_tok)

In [ ]:
# Отфильтруйте данные, где меньше трех токенов в тексте. 
data_tok = # YOUR CODE
len(data_tok)

Несколько проверок:

In [8]:
assert all(
    isinstance(row, (list, tuple)) for row in data_tok
), "please convert each line into a list of tokens (strings)"
assert all(
    all(isinstance(tok, str) for tok in row) for row in data_tok
), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all("a" <= x.lower() <= "z" for x in tok)
assert all(
    map(lambda l: not is_latin(l) or l.islower(), map(" ".join, data_tok))
), "please make sure to lowercase the data"

assert all(len(x) >= 3 for x in data_tok), \
    "Отфильтруйте данные, где меньше трех токенов в тексте"

Ниже заданы константы ширины окна контекста и проведена предобработка для построения skip-gram модели.

In [9]:
min_count = 5
window_radius = 5

Создайте словарь частот слов и множество уникальных слов, отфильтрованных по минимальной частоте.

In [10]:
# Сначала разверните список списков токенов в плоский список токенов

data_tok_1d = # YOUR CODE

# Подсчитайте частоту каждого слов. Должен получится словарь {слово:частота}
vocabulary_with_counter = # YOUR CODE

In [ ]:
# Взгляните что получилось
vocabulary_with_counter

Есть ли вам что сказать по поводу результатов? 

In [12]:
# Отфильтруйте токены, частота которых больше min_count

word_count_dict = dict()
for word, counter in vocabulary_with_counter.items():
    if # YOUR CODE

vocabulary = set(word_count_dict.keys())
del vocabulary_with_counter

In [ ]:
print(vocabulary)

In [14]:
# Создайте словарь {слово:индекс}, где индекс - это просто порядковый номер
word_to_index = # YOUR CODE

# Создайте словарь {индекс:слово} 
index_to_word = # YOUR CODE

Пары `(слово, контекст)` на основе доступного датасета сгенерированы ниже.

In [ ]:
# Cгенерируем контекстные пары для обучения Word2Vec (Skip-gram модели)

context_pairs = []

for text in data_tok:
    for i, central_word in enumerate(text):
        context_indices = range(
            # YOUR CODE,  
            # YOUR CODE
        )
        for j in context_indices:
            if j == i:
                continue
            context_word = # YOUR CODE
            if central_word in vocabulary and context_word in vocabulary:
                context_pairs.append((
                    # YOUR CODE, 
                    # YOUR CODE
                ))

print(f"Generated {len(context_pairs)} pairs of target and context words.")

In [ ]:
context_pairs[0]

### 2.2. Subsampling

Для того, чтобы сгладить разницу в частоте встречаемсости слов, необходимо реализовать механизм subsampling'а.
Для этого вам необходимо реализовать функцию ниже.

Вероятность **исключить** слово из обучения (на фиксированном шаге) вычисляется как
$$
P_\text{drop}(w_i)=1 - \sqrt{\frac{t}{f(w_i)}},
$$
где $f(w_i)$ – нормированная частота встречаемости слова, а $t$ – заданный порог (threshold).

In [17]:
def subsample_frequent_words(word_count_dict, threshold=1e-5):
    """
    Вычисляет вероятности субсэмплинга для слов на основе их частот.

    Эта функция используется для определения вероятности сохранения слова в наборе данных
    при субсэмплинге частых слов. Используемый метод вдохновлен подходом субсэмплинга
    в Word2Vec, где частота каждого слова влияет на вероятность его сохранения.

    Параметры:
    - word_count_dict (dict): Словарь, где ключи - слова, а значения - их частоты.
    - threshold (float, optional): Пороговый параметр, используемый для корректировки 
                                   частоты субсэмплинга слов. По умолчанию 1e-5.

    Возвращает:
    - dict: Словарь, где ключи - слова, а значения - вероятности сохранения каждого слова.

    Пример:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> subsample_frequent_words(word_counts)
    {'the': 0.028, 'is': 0.223, 'apple': 1.0}
    """

    total_count = # YOUR CODE 
    keep_prob_dict = {}

    for word, count in word_count_dict.items():
        f_w = # YOUR CODE 
        # Вероятность не может быть больше 1!
        p_keep = # YOUR CODE 
        keep_prob_dict[word] = # YOUR CODE
    return keep_prob_dict

In [18]:
keep_prob_dict = subsample_frequent_words(word_count_dict)
assert keep_prob_dict.keys() == word_count_dict.keys()

### 2.3. Negative sampling

Для более эффективного обучения необходимо не только предсказывать высокие вероятности для слов из контекста, но и предсказывать низкие для слов, не встреченных в контексте. Для этого вам необходимо вычислить вероятност использовать слово в качестве negative sample, реализовав функцию ниже.

В оригинальной статье предлагается оценивать вероятность слов выступать в качестве negative sample согласно распределению $P_n(w)$
$$
P_n(w) = \frac{U(w)^{3/4}}{Z},
$$

где $U(w)$ распределение слов по частоте (или, как его еще называют, по униграммам), а $Z$ – нормировочная константа, чтобы общая мера была равна $1$.

In [19]:
def get_negative_sampling_prob(word_count_dict):
    """
    Вычисляет вероятности негативного сэмплирования для слов на основе их частот.

    Эта функция корректирует частоту каждого слова, возводя ее в степень 0.75, что
    обычно используется в алгоритмах типа Word2Vec для уменьшения влияния очень частых слов.
    Затем она нормализует эти скорректированные частоты, чтобы их сумма была равна 1,
    формируя распределение вероятностей, используемое для негативного сэмплирования.

    Параметры:
    - word_count_dict (dict): Словарь, где ключи - слова, а значения - их частоты.

    Возвращает:
    - dict: Словарь, где ключи - слова, а значения - вероятности выбора каждого слова
            для негативного сэмплирования.

    Пример:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> get_negative_sampling_prob(word_counts)
    {'the': 0.298, 'is': 0.160, 'apple': 0.042}
    """

    negative_sampling_prob_dict = {}

     # 1. Вычисляем относительные частоты слов
    total_count = # YOUR CODE
    unigram_probs = # YOUR CODE

    # 2. Возводим в степень 0.75
    for word, prob in unigram_probs.items():
        negative_sampling_prob_dict[word] = # YOUR CODE

    # 3. Нормируем, чтобы сумма = 1
    Z = # YOUR CODE
    for word in negative_sampling_prob_dict:
        negative_sampling_prob_dict[word] = # YOUR CODE
    
    return negative_sampling_prob_dict

In [ ]:
negative_sampling_prob_dict = get_negative_sampling_prob(word_count_dict)
assert negative_sampling_prob_dict.keys() == negative_sampling_prob_dict.keys()

print(sum(negative_sampling_prob_dict.values()))
assert np.allclose(sum(negative_sampling_prob_dict.values()), 1)

Для удобства, преобразуем полученные словари в массивы (т.к. все слова все равно уже пронумерованы).

In [22]:
keep_prob_array = np.array(
    [keep_prob_dict[index_to_word[idx]] for idx in range(len(word_to_index))]
)

negative_sampling_prob_array = np.array(
    [
        negative_sampling_prob_dict[index_to_word[idx]]
        for idx in range(len(word_to_index))
    ]
)

Если все прошло успешно, функция ниже поможет вам с генерацией подвыборок (батчей).

In [23]:
def generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
):
    batch = []
    neg_samples = []

    while len(batch) < batch_size:
        center, context = random.choice(context_pairs)
        if random.random() < keep_prob_array[center]:
            batch.append((center, context))
            neg_sample = np.random.choice(
                range(len(negative_sampling_prob_array)),
                size=num_negatives,
                p=negative_sampling_prob_array,
            )
            neg_samples.append(neg_sample)
    batch = np.array(batch)
    neg_samples = np.vstack(neg_samples)
    return batch, neg_samples

In [24]:
batch_size = 4
num_negatives = 15
batch, neg_samples = generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
)

### 2.4. Наконец, время реализовать модель. 

- `nn.Embedding(num_embeddings, embedding_dim)` - создаёт таблицу эмбеддингов
- `nn.init.xavier_uniform_` - инициализация нормальным распределением


- `pos_scores` — скалярные dot-продукты для каждой пары (центральное слово, положительный контекст)
- `neg_scores` — dot-продукты для каждой пары (центральное слово, отрицательные контексты). Используем bmm для батчевого умножения матриц
- `torch.bmm(batch1, batch2)` — батчевое умножение матриц в PyTorch. Оно позволяет одновременно перемножать несколько пар матриц, что очень удобно для обработки батчей.

Напомним, что в случае negative sampling решается задача максимизации следующего функционала:

$$
\mathcal{L} = \log \sigma({\mathbf{v}'_{w_O}}^\top \mathbf{v}_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} \left[ \log \sigma({-\mathbf{v}'_{w_i}}^\top \mathbf{v}_{w_I}) \right],
$$

где:
- $\mathbf{v}_{w_I}$ – вектор центрального слова $w_I$,
- $\mathbf{v}'_{w_O}$ – вектор слова из контекста $w_O$,
- $k$ – число negative samplesЮ,
- $P_n(w)$ – распределение negative samples, заданное выше,
- $\sigma$ – сигмоида.

`nn.BCEWithLogitsLoss()` – это функция потерь в PyTorch, которая объединяет Sigmoid активацию + BCELoss (Binary Cross-Entropy) в одной оптимизированной функции. Используется для бинарной классификации (2 класса)

$loss = - \left[y \cdot \log(\sigma(x)) + (1-y) \cdot \log(1-\sigma(x)) \right] = $

In [25]:
class SkipGramModelWithNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.center_embeddings = None  # YOUR CODE HERE
        self.context_embeddings = None  # YOUR CODE HERE

    def forward(self, center_words, pos_context_words, neg_context_words):
        # YOUR CODE HERE
        pos_scores = 0  
        neg_scores = 0 

        return pos_scores, neg_scores

In [27]:
device = torch.device("cpu")

In [28]:
vocab_size = len(word_to_index)
embedding_dim = 32
num_negatives = 15

model = SkipGramModelWithNegSampling(vocab_size, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05)
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=150)

criterion = nn.BCEWithLogitsLoss()

In [29]:
params_counter = 0
for weights in model.parameters():
    params_counter += weights.shape.numel()
assert params_counter == len(word_to_index) * embedding_dim * 2

In [30]:
def train_skipgram_with_neg_sampling(model,context_pairs,keep_prob_array,word_to_index,batch_size,num_negatives, negative_sampling_prob_array,steps,optimizer=optimizer,lr_scheduler=lr_scheduler,device=device,):
    
    pos_labels = torch.ones(batch_size).to(device)
    neg_labels = torch.zeros(batch_size, num_negatives).to(device)
    loss_history = []
    for step in tqdma(range(steps)):

        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs, batch_size, keep_prob_array, word_to_index,
            num_negatives, negative_sampling_prob_array)
        
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(device)
        pos_context_words = torch.tensor([pair[1] for pair in batch], dtype=torch.long).to(device)
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)

        optimizer.zero_grad()

        pos_scores, neg_scores = model(center_words, pos_context_words, neg_context_words)

        loss_pos = criterion(pos_scores, pos_labels)
        loss_neg = criterion(neg_scores, neg_labels)

        loss = loss_pos + loss_neg
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        lr_scheduler.step(loss_history[-1])

        if step % 100 == 0:
            print(
                f"Step {step}, Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )

In [31]:
def train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    device=device,
):
    pos_labels = torch.ones(batch_size).to(device)
    neg_labels = torch.zeros(batch_size, num_negatives).to(device)
    loss_history = []
    for step in tqdma(range(steps)):

        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs, batch_size, keep_prob_array, word_to_index,
            num_negatives, negative_sampling_prob_array)
        
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(device)
        pos_context_words = torch.tensor([pair[1] for pair in batch], dtype=torch.long).to(device)
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)

        optimizer.zero_grad()

        pos_scores, neg_scores = model(center_words, pos_context_words, neg_context_words)

        loss_pos = criterion(pos_scores, pos_labels)
        loss_neg = criterion(neg_scores, neg_labels)

        loss = loss_pos + loss_neg
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        lr_scheduler.step(loss_history[-1])

        if step % 100 == 0:
            print(
                f"Step {step}, Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )

In [ ]:
steps = 2500
batch_size = 512
train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
)

Наконец, используйте полученную матрицу весов в качестве матрицы в векторными представлениями слов. Рекомендуем использовать для сдачи матрицу, которая отвечала за слова из контекста (т.е. декодера).

In [114]:
_model_parameters = model.parameters()

# Предполагаем, что первая матрица была предназначена для центрального слова
embedding_matrix_center = next(_model_parameters).detach()  

# Предполагаем, что вторая матрица была предназначена для контекстного слова
embedding_matrix_context = next(_model_parameters).detach()  

In [115]:
def get_word_vector(word, embedding_matrix, word_to_index=word_to_index):
    return embedding_matrix[word_to_index[word]]

Простые проверки:

In [ ]:
similarity_1 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("apple", embedding_matrix_context)[None, :],
)

similarity_2 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("dell", embedding_matrix_context)[None, :],
)

print(f"similarity_1 = {similarity_1}")
print(f"similarity_2 = {similarity_2}")

assert similarity_1 > similarity_2

In [ ]:
similarity_1 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("laptop", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("macbook", embedding_matrix_context)[None, :],
)

print(f"similarity_1 = {similarity_1}")
print(f"similarity_2 = {similarity_2}")

assert similarity_1 > similarity_2

Наконец, взглянем на ближайшие по косинусной мере слова. Функция реализована ниже.

In [118]:
def find_nearest(word, embedding_matrix, word_to_index=word_to_index, k=10):
    word_vector = get_word_vector(word, embedding_matrix)[None, :]
    dists = F.cosine_similarity(embedding_matrix, word_vector)
    index_sorted = torch.argsort(dists)
    top_k = index_sorted[-k:]
    return [(index_to_word[x], dists[x].item()) for x in top_k.numpy()]

In [ ]:
find_nearest("python", embedding_matrix_context, k=10)

Также вы можете визуально проверить, как представлены в латентном пространстве часто встречающиеся слова.

In [120]:
top_k = 5000
_top_words = sorted([x for x in word_count_dict.items()], key=lambda x: x[1])[
    -top_k - 100 : -100
]  # ignoring 100 most frequent words
top_words = [x[0] for x in _top_words]
del _top_words

In [121]:
word_embeddings = torch.cat(
    [embedding_matrix_context[word_to_index[x]][None, :] for x in top_words], dim=0
).numpy()

In [ ]:
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook

output_notebook()


def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs,
):
    """draws an interactive plot for data points with auxilirary info on hover"""
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

In [123]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_embeddings)

In [ ]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=top_words)

In [ ]:
# do not change the code in the block below
# __________start of block__________
import os
import json

assert os.path.exists(
    "words_subset.txt"
), "Please, download `words_subset.txt` and place it in the working directory"

with open("words_subset.txt") as iofile:
    selected_words = iofile.read().split("\n")


def get_matrix_for_selected_words(selected_words, embedding_matrix, word_to_index):
    word_vectors = []
    for word in selected_words:
        index = word_to_index.get(word, None)
        vector = [0.0] * embedding_matrix.shape[1]
        if index is not None:
            vector = embedding_matrix[index].numpy().tolist()
        word_vectors.append(vector)
    return word_vectors


word_vectors = get_matrix_for_selected_words(
    selected_words, embedding_matrix_context, word_to_index
)

with open("submission_dict.json", "w") as iofile:
    json.dump(word_vectors, iofile)
print("File saved to `submission_dict.json`")
# __________end of block__________